<a href="https://colab.research.google.com/github/Aadhil21/DSGP-Group28/blob/branch-Aadhil/Extract_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [10]:
import pandas as pd
import cv2
import os
import mediapipe as mp
import time

#Initialize mediapipe model

In [11]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(min_detection_confidence=0.6,
                    min_tracking_confidence=0.6,
                    static_image_mode=False,
                    smooth_landmarks=True,
                    model_complexity=2)

##Set Directory Path

In [12]:
# if not os.path.exists('Videos'):
#     os.mkdir('Videos')
# if not os.path.exists('Datasets'):
#     os.mkdir('Datasets')

video_name = 'WPushUp.mp4'
video_path = 'Videos/' + video_name
dataset_path = 'Datasets/'

In [13]:
frame_count = 0
frame_numbers = []
data_dict = {}
desired_width = 480
desired_height = 360

In [14]:
cap = cv2.VideoCapture(video_path)

##Populate keypoints dictionary

In [15]:
for landmark in mp_pose.PoseLandmark:
    x_list_name = f"{landmark.name}_Point_x"
    y_list_name = f"{landmark.name}_Point_y"
    z_list_name = f"{landmark.name}_Point_z"
    visibility_list_name = f"{landmark.name}_visibility"

    data_dict[x_list_name] = []
    data_dict[y_list_name] = []
    data_dict[z_list_name] = []
    data_dict[visibility_list_name] = []

##Process Frames

In [16]:
while cap.isOpened():
    success, image = cap.read()
    if not success:
        break

    # Increment frame_count
    frame_count += 1
    print(f'Processing frame {frame_count}')

    # Resize the frame to the desired width and height
    image = cv2.resize(image, (desired_width, desired_height))

    # Convert the image from BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = pose.process(image_rgb)

    if results.pose_landmarks:

        frame_numbers.append(frame_count)

        for landmark in mp_pose.PoseLandmark:
            x_list_name = f"{landmark.name}_Point_x"
            y_list_name = f"{landmark.name}_Point_y"
            z_list_name = f"{landmark.name}_Point_z"
            visibility_list_name = f"{landmark.name}_visibility"

            data_dict[x_list_name].append(results.pose_landmarks.landmark[landmark].x)
            data_dict[y_list_name].append(results.pose_landmarks.landmark[landmark].y)
            data_dict[z_list_name].append(results.pose_landmarks.landmark[landmark].z)
            data_dict[visibility_list_name].append(results.pose_landmarks.landmark[landmark].visibility)

    time.sleep(0.01)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Processing frame 1


C:\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39
Processing frame 40
Processing frame 41
Processing frame 42
Processing frame 43
Processing frame 44
Processing frame 45
Processing frame 46
Processing frame 47
Processing frame 48
Processing frame 49
Processing frame 50
Processing frame 51
Processi

## Save Key points to File

In [17]:
# Creating a DataFrame from the dictionary
df = pd.DataFrame(data_dict)

# Adding a column for the frame number
df.insert(0, 'Frame_Number', frame_numbers)

# Writing the DataFrame to an Excel file
file_name = 'pose_landmark_data_' + video_name.split('.')[0] + '.xlsx'

df.to_excel(dataset_path + '/' + file_name, index=False)

print(f"Data saved to {file_name}")

Data saved to pose_landmark_data_WPushUp.xlsx


## Reading Key points from File

In [18]:
df_excel = pd.read_excel('Datasets/' + file_name)
df_excel

,Frame_Number,NOSE_Point_x,NOSE_Point_y,NOSE_Point_z,NOSE_visibility,LEFT_EYE_INNER_Point_x,LEFT_EYE_INNER_Point_y,LEFT_EYE_INNER_Point_z,LEFT_EYE_INNER_visibility,LEFT_EYE_Point_x,...,RIGHT_HEEL_Point_z,RIGHT_HEEL_visibility,LEFT_FOOT_INDEX_Point_x,LEFT_FOOT_INDEX_Point_y,LEFT_FOOT_INDEX_Point_z,LEFT_FOOT_INDEX_visibility,RIGHT_FOOT_INDEX_Point_x,RIGHT_FOOT_INDEX_Point_y,RIGHT_FOOT_INDEX_Point_z,RIGHT_FOOT_INDEX_visibility
0,1,0.254621,0.302891,-0.016454,0.999914,0.249138,0.278250,-0.045513,0.999861,0.250954,...,0.291700,0.055152,0.934912,0.507480,-0.007927,0.258004,0.920347,0.474278,0.232524,0.035637
1,2,0.255304,0.302962,-0.039610,0.999917,0.249683,0.278400,-0.072141,0.999866,0.251444,...,0.407340,0.057414,0.937241,0.508591,0.100030,0.262736,0.933384,0.473314,0.342932,0.037212
2,3,0.255342,0.306269,-0.021503,0.999911,0.250044,0.280966,-0.053738,0.999859,0.251828,...,0.379696,0.055038,0.939340,0.505883,0.085299,0.256015,0.935431,0.469985,0.320846,0.035728
3,4,0.251655,0.308809,0.025822,0.999917,0.247719,0.285840,0.000819,0.999869,0.249826,...,0.219868,0.052821,0.915376,0.534277,-0.022528,0.253592,0.913949,0.472692,0.162995,0.034842
4,5,0.247827,0.312926,0.020259,0.999909,0.244061,0.291691,0.003745,0.999864,0.246229,...,0.193430,0.052662,0.304477,0.342846,-0.202465,0.239753,0.344384,0.346067,0.161913,0.034747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,246,0.224735,0.327147,0.211180,0.898402,0.215912,0.316137,0.194252,0.900432,0.216361,...,0.045927,0.286617,0.263923,0.572164,-0.068789,0.364256,0.338562,0.379395,0.021906,0.213263
223,248,0.276313,0.364584,0.453348,0.820865,0.271730,0.358414,0.444885,0.825158,0.270966,...,-0.096089,0.258070,0.203008,0.380190,-0.420590,0.327936,0.378837,0.362017,-0.135097,0.191961
224,249,0.246253,0.340955,0.225557,0.838225,0.237305,0.323673,0.214109,0.842355,0.237455,...,-0.038515,0.252789,0.310586,0.501042,0.041617,0.297581,0.448940,0.384846,-0.056386,0.184980
225,250,0.243613,0.339273,0.191251,0.853815,0.233871,0.321845,0.173282,0.857851,0.233878,...,0.041984,0.235594,0.309524,0.540204,0.155494,0.269453,0.378085,0.394025,0.036455,0.170082
